In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Solving the Schrodinger equation ##

Here we give an example of numerically solving the Schrodinger equation using [Numerov's method](https://en.wikipedia.org/wiki/Numerov%27s_method).  We'll do the harmonic oscillator, which is something we can do entirely without numerical methods, just because it is so familiar.  We will use "natural" units ($\hbar=m=1$) throughout.

In [ ]:
# Define the problem space:
Lside = 2.0   # The length of the computation space, in natural units.
N     = 2048  # The size of the grid.
TOL   = 1e-4  # A numerical tolerance for convergence.

# Set up some "helper" functions.
def V(x):
    """Our potential -- a hamonic oscillator."""
    return( 0.5*x**2 )

def posn(n):
    """Convert from grid position to x coordinate."""
    return( -Lside/2.0 + n*Lside/N )

hh = posn(1) - posn(0)
h2 = hh**2

def keff2(E,x):
    return( 2*(E-V(x)) )

def psiup(n,E,psi1,psi2):
    """Compute Psi_n using the Numerov algorithm.  The second
    derivative is approximated by finite difference but the
    Psi^{(4)} correction term is included using the 2nd derivative
    of the Schrodinger equation."""
    t    = 1./12.0
    knm0 = keff2(E,posn(n-0))
    knm1 = keff2(E,posn(n-1))
    knm2 = keff2(E,posn(n-2))
    psi  = 2*(1-5*t*h2*knm1)*psi1 - (1+t*h2*knm2)*psi2
    psi /= 1+t*h2*knm0
    return(psi)

In [ ]:
# Code the Numerov solver.
def numerov_solve(E):
    """Does Numerov until convergence.  This is coded very non-Pythonically,
    but hopefully in a way that is clear."""
    psi    = np.zeros(N+1)
    psi[0] = 0.0
    psi[1] = 1e-10
    for n in range(2,N+1):
        psi[n] = psiup(n,E,psi[n-1],psi[n-2])
    return( (np.max(psi),psi) )

In [ ]:
# Now do an example of finding a solution for the energy eigenvalue
# using a simple bisection algorithm.
Emin,Emax = 0.0,2.0   # We know the energy is in this range.
keepgoing = True
while keepgoing:
    E          = 0.5*(Emin+Emax)
    psimax,psi = numerov_solve(E)
    print("E={:12.4e}, Psi_max={:12.4e}, Psi[-1]={:12.4e} Psi_max".\
          format(E,psimax,psi[-1]/psimax))
    if psi[-1]<0:
        Emax = E
    elif psi[-1]>0:
        Emin = E
    keepgoing = np.abs(psi[-1])>TOL*psimax